In [8]:
import numpy as np 
import torch
import torchvision 
import torch.nn as nn
import torch.nn.functional as f
torch.__version__

'1.12.1+cu116'

In [9]:
from torchvision import datasets
import torchvision.transforms as transforms 
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

train_dataset = datasets.CIFAR10(root="./", train=True, download=False, transform=transform)
test_dataset = datasets.CIFAR10(root="./", train=False, download=False, transform=transform)
print(train_dataset)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [10]:
train_dataset[0][0].size()

torch.Size([3, 32, 32])

In [11]:
class ConvNet(nn.Module): 
  def __init__(self, output_dim): 
    super().__init__() 

    #two layesr for convolution 
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1) 
    self.maxPool1 = nn.MaxPool2d(kernel_size=2) #32x32 -> 16x16

    self.conv2 = nn.Conv2d(64, 10, kernel_size=3, padding=1)
    self.maxPool2 = nn.MaxPool2d(kernel_size=2) #16x16 -> 8x8

    #two fully connected layers 
    self.fc1 = nn.Linear(640, 2080) #8x8x10 = 640
    self.fc2 = nn.Linear(2080, output_dim)
    
  def forward(self, x): #forward prop 
    x= f.relu(self.conv1(x)) 
    x = self.maxPool1(x)
    x= f.relu(self.conv2(x)) 
    x = self.maxPool2(x)

    x = torch.flatten(x, 1)

    x = f.relu(self.fc1(x))
    x = self.fc2(x)

    return x 

In [12]:
DEVICE = torch.device("cuda") # for using gpu 
OUTPUT_DIM = 10 
LEARNING_RATE = 0.001
NUM_EPOCHS = 5 
print(DEVICE)

cuda


In [13]:
model = ConvNet(OUTPUT_DIM).to(DEVICE) 
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [15]:
def eval_accuracy(loader): 
  model.eval() #set model to inference 
  yTrue = []
  yPred = []

  count = 0 
  accuracy_sum = 0 

  with torch.no_grad(): 
    for x, y, in loader: 
      x = x.float().to(DEVICE)
      y = y.to(DEVICE)
      outputs = model.forward(x)

      outputs = outputs.cpu().numpy() 
      yPred = np.argmax(outputs, axis=1)
      y = y.numpy()

      match = (y == yPred).astype('uint8')
      accuracy_sum += np.sum(match)
      count += len(match)

  accuracy = accuracy_sum/count 
  return accuracy 

In [17]:
model.train() #Sets the module in training mode.

for i in range(NUM_EPOCHS): 
  loss_record = []

  # x=input image y=label 
  for(x, y) in train_loader: 
    x = x.float().to(DEVICE) 
    y = y.to(DEVICE)

    outputs = model.forward(x)
    loss = criterion(outputs, y) 
    loss_record.append(loss.item())

    optimizer.zero_grad() #reset gradient for each batch 
    loss.backward() 
    optimizer.step() 
  
  print("Loss at epoch %d: %.4f" %(i+1, np.mean(loss_record)))
  train_accuracy = eval_accuracy(train_loader)
  test_accuracy = eval_accuracy(test_loader)
  print("train accuracy: %.4f" %(train_accuracy))
  print("test accuracy: %.4f" %(test_accuracy))


Loss at epoch 1: 1.3933
train accuracy: 0.5192
test accuracy: 0.5116
Loss at epoch 2: 1.3288
train accuracy: 0.5342
test accuracy: 0.5218
Loss at epoch 3: 1.2818
train accuracy: 0.5572
test accuracy: 0.5433
Loss at epoch 4: 1.2428
train accuracy: 0.5722
test accuracy: 0.5527
Loss at epoch 5: 1.2011
train accuracy: 0.5778
test accuracy: 0.5562
